In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint

headers = {"Accept-Language": "en-US, en;q=0.5"}

name = []
year = []
duration = []
rating= []
description = []
director = []
stars = []
reviews = []

pages = np.arange(1, 10000, 100)

for page in pages: 
    page = requests.get("https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,2023-05-01&languages=en&runtime=1,10000&num_votes=1,100000000000000&sort=release_date,desc&count=100&start=" + str(page) + "&ref_=adv_nxt")
    soup = BeautifulSoup(page.text, "html.parser")
    movie_data = soup.findAll('div', attrs = {'class' : 'lister-item mode-advanced'})
    sleep(randint(2, 8))

    for store in movie_data:
        title = store.h3.a.text

        describe = store.find_all('p', class_ = 'text-muted')
        description_ = describe[1].text.replace('\n', '') if len(describe) >1 else '*****'
        if description_ == 'Add a Plot':
            continue
        description.append(description_)

        name.append(title)
        
        year_of_release = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text.split(' ')
        year_of_release = year_of_release[-1].replace('(', '').replace(')', '')
        year.append(year_of_release)
        
        runtime = store.p.find("span", class_ = 'runtime').text.replace(' min', '') if store.p.find("span", class_ = 'runtime') else "***"
        duration.append(runtime)
        
        rate = store.find('div', class_ = "inline-block ratings-imdb-rating").text.replace('\n', '') if store.find('div', class_ = "inline-block ratings-imdb-rating") else "****"
        rating.append(rate)
        

        cast = store.find("p", class_ = '')
        cast = cast.text.replace('\n', '').split('|')
        cast = [x.strip() for x in cast]
        if len(cast) == 2:
            if "Director:" in cast[0]:
                cast = [cast[0].replace("Director:", ""), cast[1].replace("Stars:", "")]
            elif "Directors:" in cast[0]:
                cast = [cast[0].replace("Directors:", ""), cast[1].replace("Stars:", "")]
        elif len(cast) == 1:
            if "Director:" in cast[0]:
                cast = [cast[0].replace("Director:", ""), "NA"]
            elif "Directors:" in cast[0]:
                cast = [cast[0].replace("Directors:", ""), "NA"]
            else:
                cast = ["NA", cast[0].replace("Stars:", "")]
        else:
            cast = ["NA", "NA"]
        director.append([x.strip() for x in cast[0].split(",")])
        stars.append([x.strip() for x in cast[1].split(",")])

        # Scrape Reviews by entering the movie page
        movie_page = requests.get("https://www.imdb.com" + store.h3.a['href'] + "reviews?ref_=tt_urv")
        soup = BeautifulSoup(movie_page.text, "html.parser")
        review_data = soup.findAll('div', attrs = {'class' : 'lister-item mode-detail imdb-user-review collapsable'})
        sleep(randint(2, 8))
        review_array = []

        for review in review_data:
            review_text = review.find('div', class_ = 'text show-more__control').text.replace('\n', '') if review.find('div', class_ = 'text show-more__control') else '*****'
            review_text = review_text.replace('*** This review may contain spoilers ***', '')
            review_array.append(review_text)
        reviews.append(review_array)
          

In [3]:
#creating a dataframe 
movie_list = pd.DataFrame({ "Movie Name": name, "Year of Release" : year, "Watch Time": duration, "Movie Rating": rating, "Description": description, "Director": director, "Stars": stars, "Reviews": reviews})

In [ ]:
movie_list

In [4]:
import csv
movie_list.to_csv("Movie Dataset.csv")